In [1]:
from box import ConfigBox

In [2]:
d={'key':'value','key1':'value1'}
d['key']

'value'

In [3]:
d.key

AttributeError: 'dict' object has no attribute 'key'

In [7]:
d2=ConfigBox(d)
d2.key1

'value1'

In [8]:
from ensure import ensure_annotations